# LFMC Projection - Mapping Models
Train Nowcasting and 3-month projection models to create LFMC maps

In [ ]:
import os
import json
import numpy as np
import pandas as pd

import initialise
import common
from modelling_functions import create_models, run_experiment
from architecture_projection import model_params
from model_parameters import ExperimentParams

## Directories and Input files
Change these settings as required
- `input_dir`: Directory containing the data extracted from GEE and Globe-LFMC, the outputs from running the `Extract DEM Data.ipynb` and `Extract MODIS Data.ipynb` notebooks.
- `modis_csv`: The file containing extracted MODIS data for each sample, created by `Extract MODIS Data.ipynb`
- `prism_csv`: The file containing extracted PRISM data for each sample, created by `Extract PRISM Data.ipynb`
- `aux_csv`: The file containing extracted sample labels, DEM, climate zone and other auxiliary data, created by `Extract Auxiliary Data.ipynb`.

In [ ]:
modis_csv = os.path.join(common.DATASETS_DIR, 'modis_730days.csv')
prism_csv = os.path.join(common.DATASETS_DIR, 'prism_730days.csv')
aux_csv = os.path.join(common.DATASETS_DIR, 'samples_730days.csv')

## Set up experiment parameters
If the experiment dictionary contains a 'tests' key that is not 'falsy' (False, None, 0, empty list) it is assumed to be a list of tests to run. Each test will run with the specified model parameters. Model parameters not specified will be the same for each test, as set in the main model_params dictionary. A failed run can be restarted by setting the 'restart' key to the test that failed. This test and the remaining tests will then be run.

In [ ]:
experiment = ExperimentParams({
    'name': 'final_models',
    'description': 'Nowcasting and 3-month projection models for the LFMC maps',
    'tests': [
        {'testName': 'Nowcasting',
         'inputs': {'optical': {'start': -365, 'end': 0},
                    'weather': {'start': -365, 'end': 0}}},
        {'testName': '3-months lead time',
         'inputs': {'optical': {'start': -275, 'end': 90},
                    'weather': {'start': -275, 'end': 90}}},
    ],
    'restart': None,
})
experiment

## Set up model parameters
Set up and customise the model parameters. Leave all parameters as set here to run Scenario A. To find out more about any parameter, run `model_params.help('<parameter>')` after running this cell to create the ModelParams object.

In [ ]:
# Customize model parameters
model_params['modelName'] = experiment['name']
model_params['description'] = experiment['description']
model_params['samplesFile'] = aux_csv
model_params['modelRuns'] = common.ENSEMBLE_SIZE
model_params['tempDir'] = common.TEMP_DIR
model_params['modelDir'] = os.path.join(common.MODELS_DIR, model_params['modelName'])
model_params['derivedModels'] = common.DERIVED_MODELS
model_params['seedList'] = [
    566, 451, 795, 237, 788, 185, 397, 530, 758, 633,
    914, 326, 334, 366, 336, 413, 111, 599, 416, 230,
]

# Exclude 2018 data from training samples to ensure we don't use any data from the 3-month lead time
model_params['yearColumn'] = 'Sampling year'
model_params['splitMethod'] = 'byYear'
model_params['splitYear'] = 2018

model_params['saveModels'] = True
model_params['gpuDevice'] = 0

model_params

In [ ]:
model_params.add_input('optical', {'filename': modis_csv, 'channels': 7})
model_params.add_input('weather', {'filename': prism_csv, 'channels': 7})
model_params['inputs']

## Build and run the models
Builds and trains the LFMC models.

All models, predictions, evaluation statistics, and plots of test results are saved to `model_dir`, with each test and run saved to a separate sub-directory. For each model created, predictions and evaluation statistics are also returned as attributes of the `model` object. These are stored as nested lists, the structure for a full experiment is:
- Tests (omitted if not an experiment)
  - Runs (omitted for a single run)
    - Folds (for k-fold splitting)

In [ ]:
models = run_experiment(experiment, model_params)
for model in models:
    display(getattr(model, 'test_stats', None))